In [1]:
import pandas as pd
import numpy as np
import os
import re

In [33]:
behavioral_risk_file = 'BRFSS_2014_Overall.csv'
census_directory = 'aff_download'
output_directory = 'data_cleaned'
# census data files
geographic_mobility = 'ACS_14_1YR_B07001_with_ann.csv'
home_language = 'ACS_14_1YR_S1601_with_ann.csv'
mean_income = 'ACS_14_1YR_S1902_with_ann.csv'
median_income = 'ACS_14_1YR_S1903_with_ann.csv'

# select census file
selected_file = home_language

# import files
risk_data = pd.read_csv(behavioral_risk_file)
compare = pd.read_csv(os.path.join(census_directory,selected_file))
for column in compare.keys():
    compare[column] = pd.to_numeric(compare[column], errors='ignore')

In [3]:
compare.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC01_EST_VC02,...,HC02_EST_VC36,HC02_MOE_VC36,HC03_EST_VC36,HC03_MOE_VC36,HC01_EST_VC37,HC01_MOE_VC37,HC02_EST_VC37,HC02_MOE_VC37,HC03_EST_VC37,HC03_MOE_VC37
0,Id,Id2,Geography,Total; Estimate; Population 5 years and over,Total; Margin of Error; Population 5 years and...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Total; Estimate; Speak only English,...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Total; Estimate; PERCENT IMPUTED - Ability to ...,Total; Margin of Error; PERCENT IMPUTED - Abil...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...,Percent of specified language speakers - Spea...
1,0400000US01,01,Alabama,4558217,2782,98.0,0.1,2.0,0.1,95.1,...,(X),(X),(X),(X),6.4,(X),(X),(X),(X),(X)
2,0400000US02,02,Alaska,683324,986,94.9,0.5,5.1,0.5,83.4,...,(X),(X),(X),(X),4.7,(X),(X),(X),(X),(X)
3,0400000US04,04,Arizona,6298821,1394,90.7,0.2,9.3,0.2,73.0,...,(X),(X),(X),(X),8.4,(X),(X),(X),(X),(X)
4,0400000US05,05,Arkansas,2775973,2624,96.7,0.2,3.3,0.2,93.1,...,(X),(X),(X),(X),3.9,(X),(X),(X),(X),(X)


In [4]:
def replace_str(x):
    try:
        float(x)
        return float(x)
    except:
        return np.NaN
data_cols = compare.keys()[5:]
for col in data_cols:
    compare[col][1:] = compare[col][1:].map(replace_str)
compare_clean = compare.dropna(axis=1,how='any').drop(columns=['GEO.id','GEO.id2']).set_index('GEO.display-label')

In [5]:
risk_questions = risk_data['Question'].unique()
print('Number of questions in Risk file to compare: %s' % len(risk_questions))

Number of questions in Risk file to compare: 64


In [6]:
filtered_questions = []
for n,q in enumerate(risk_questions):
    if re.search('aged',q) or re.search('race',q):
        print(f'{n}: {q}')
    else:
        filtered_questions.append(q)

print('')
for n,q in enumerate(filtered_questions):
    print(f'{n}: {q}')

0: What is your race/ethnicity?
17: Adults aged 50+ who have had a blood stool test within the past two years (variable calculated from one or more BRFSS questions)
18: Adults aged 50-75 who have had a blood stool test within the past year (variable calculated from one or more BRFSS questions)
19: Adults aged 50+ who have ever had a sigmoidoscopy or colonoscopy (variable calculated from one or more BRFSS questions)
20: Respondents aged 50-75 who have fully met the USPSTF recommendation (variable calculated from one or more BRFSS questions)
41: Adults aged 18-64 who have any kind of health care coverage (variable calculated from one or more BRFSS questions)
45: Adults aged 65+ who have had a flu shot within the past year (variable calculated from one or more BRFSS questions)
46: Adults aged 65+ who have ever had a pneumonia vaccination (variable calculated from one or more BRFSS questions)
50: Adults aged 65+ who have had all their natural teeth extracted (variable calculated from one o

In [7]:
for column in compare_clean.keys():
    compare_clean[column] = pd.to_numeric(compare_clean[column] , errors = 'coerce')
compare_clean.describe()

,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC01_EST_VC02,HC01_MOE_VC02,HC01_EST_VC03,HC01_MOE_VC03,...,HC03_MOE_VC31,HC01_EST_VC32,HC01_MOE_VC32,HC02_EST_VC32,HC02_MOE_VC32,HC03_EST_VC32,HC03_MOE_VC32,HC01_EST_VC35,HC01_EST_VC36,HC01_EST_VC37
count,5.200000e+01,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,...,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000
mean,5.816311e+06,1892.884615,93.023077,0.223077,6.976923,0.223077,83.892308,0.344231,16.107692,0.344231,...,3.128846,4.940385,0.226923,70.903846,2.876923,29.096154,2.876923,6.007692,5.590385,6.248077
std,6.610916e+06,972.895288,10.864236,0.135206,10.864236,0.135206,14.800051,0.197448,14.800051,0.197448,...,2.103250,3.759133,0.147017,6.689616,1.979465,6.689616,1.979465,0.931969,1.432981,1.458107
min,5.462240e+05,6.000000,21.600000,0.100000,0.600000,0.100000,5.500000,0.100000,2.000000,0.100000,...,0.300000,0.100000,0.100000,53.600000,0.500000,12.000000,0.500000,3.800000,2.500000,3.100000
25%,1.691520e+06,1217.750000,91.825000,0.100000,2.775000,0.100000,78.800000,0.200000,6.975000,0.200000,...,1.700000,2.175000,0.100000,67.775000,1.475000,25.725000,1.475000,5.400000,4.800000,5.400000
50%,3.941298e+06,1874.500000,95.550000,0.200000,4.450000,0.200000,88.850000,0.300000,11.150000,0.300000,...,2.550000,4.100000,0.200000,69.950000,2.250000,30.050000,2.250000,5.900000,5.400000,6.100000
75%,6.438912e+06,2551.750000,97.225000,0.225000,8.175000,0.225000,93.025000,0.400000,21.200000,0.400000,...,4.375000,6.675000,0.300000,74.275000,3.725000,32.225000,3.725000,6.525000,6.200000,6.900000
max,3.629081e+07,4957.000000,99.400000,0.600000,78.400000,0.600000,98.000000,0.900000,94.500000,0.900000,...,7.900000,18.700000,0.700000,88.000000,11.800000,46.400000,11.800000,8.900000,11.100000,11.800000


In [8]:
# for counter,question in enumerate(risk_questions):
#     print(f'{counter}: {question}')

In [28]:
n_per_q = 8
results = pd.DataFrame({'question':[],'combination':[],'R':[]})
for counter,question in enumerate(filtered_questions):
    this_question = risk_data.loc[risk_data['Question']==question].pivot(
        index = 'Locationdesc',columns = 'Response',values = 'Data_value')
    question_responses = list(this_question.keys())
    combined = this_question.merge(compare_clean.filter(
        regex='HC01_EST_VC0'), right_index=True, left_index=True, how='inner')
    compare_slices = list(compare_clean.filter(regex='HC01_EST_VC0').keys())

    correlation_matrix = combined.corr()
    corr_series = {}
    for this_slice in compare_slices:
        for response in question_responses:
    #         print(f'{this_slice},{response}:{correlation_matrix[this_slice][response]}')
            corr_series[f'{this_slice},{response}'] = abs(correlation_matrix[this_slice][response])
    corr_series = pd.Series(corr_series)
    corr_series.sort_values(ascending=False)
#     print(f'{counter}: {question}')
#     print(corr_series.sort_values(ascending=False).head(1))
    results = results.append(pd.DataFrame({'question': [question for i in range(n_per_q)], 
                                 'combination':corr_series.sort_values(ascending=False).head(n_per_q).index,
                                 'R':corr_series.sort_values(ascending=False).head(n_per_q).values
                                },index = [i for i in range(n_per_q*(counter),n_per_q*(counter+1))]))
#     print(results)
results.sort_values('R',ascending=False).head()

,R,combination,question
208,0.822810,"HC01_EST_VC04,Less than $15,000",What is your annual household income?
200,0.773962,"HC01_EST_VC04,Homemaker",What is your employment status?
96,0.761591,"HC01_EST_VC06,No, pre-diabetes or borderline d...",Have you ever been told by a doctor that you h...
296,0.747801,"HC01_EST_VC04,Fair",How is your general health?
209,0.710191,"HC01_EST_VC03,Less than $15,000",What is your annual household income?


In [19]:
top = results.sort_values('R',ascending=False).head(n_per_q*10)
top

,R,combination,question
208,0.822810,"HC01_EST_VC04,Less than $15,000",What is your annual household income?
200,0.773962,"HC01_EST_VC04,Homemaker",What is your employment status?
96,0.761591,"HC01_EST_VC06,No, pre-diabetes or borderline d...",Have you ever been told by a doctor that you h...
296,0.747801,"HC01_EST_VC04,Fair",How is your general health?
209,0.710191,"HC01_EST_VC03,Less than $15,000",What is your annual household income?
210,0.710191,"HC01_EST_VC02,Less than $15,000",What is your annual household income?
297,0.706037,"HC01_EST_VC04,Very good",How is your general health?
376,0.690721,"HC01_EST_VC05,Every day","Do you currently use chewing tobacco, snuff, o..."
211,0.687135,"HC01_EST_VC03,$35,000 - 49,999",What is your annual household income?
212,0.687135,"HC01_EST_VC02,$35,000 - 49,999",What is your annual household income?


In [20]:
list(top['combination'].map(lambda x: x[:13]).unique())

['HC01_EST_VC04',
 'HC01_EST_VC06',
 'HC01_EST_VC03',
 'HC01_EST_VC02',
 'HC01_EST_VC05']

In [21]:
compare[top['combination'].map(lambda x: x[:13]).mode()[0]][0]

'Total; Estimate; Speak a language other than English - Spanish or Spanish Creole'

In [32]:
select_questions = list(top.groupby(['question']).count().sort_values(['R'],ascending=False).head(4).index)
for n,q in enumerate(select_questions):
    print(f'{n+1}: {q}')

1: What is your annual household income?
2: How is your general health?
3: Do you currently use chewing tobacco, snuff, or snus?
4: What is your employment status?


In [92]:
# Output to csv
question_map = {}
compare.filter(regex='HC01_EST_VC0').loc[0].map(lambda x: x[17:]).to_csv(os.path.join(output_directory,'demographics_map.csv'))
names = compare.filter(regex='HC01_EST_VC0').loc[0].map(lambda x: x[17:]).to_dict()
for counter,question in enumerate(select_questions):
    question_map[f'Q{counter+1}']=question
    this_question = risk_data.loc[risk_data['Question']==question].pivot(
        index = 'Locationdesc',columns = 'Response',values = 'Data_value')
    answer_map = {}
    for index,response in enumerate(risk_data.loc[(risk_data['Question']==question,'Response')].unique()):
        answer_map[f'A{index+1}'] = response
    pd.Series(answer_map).to_csv(os.path.join(output_directory,f'Q{counter+1}_answer_map.csv'))
    this_question.rename({value:key for key,value in answer_map.items()},axis='columns',inplace=True)
    question_responses = list(this_question.keys())
    combined = this_question.merge(compare_clean.filter(
        regex='HC01_EST_VC0'), right_index=True, left_index=True, how='inner')
#     combined.rename(names, axis='columns', inplace=True)
    combined.to_csv(os.path.join(output_directory,f'Q{counter+1}.csv'))
pd.Series(question_map).to_csv(os.path.join(output_directory,'question_map.csv'))